### Import required packages

In [ ]:
from resemblyzer import VoiceEncoder, preprocess_wav
import numpy as np
from umap import UMAP
import matplotlib.pyplot as plt
import librosa
import os
import sys
import pickle as pkl
import pyaudio
import wave
import struct

### To enable microphone input

In [ ]:
fs=16000
FRAMESIZE = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 10
NOFRAMES = int(RATE*RECORD_SECONDS // FRAMESIZE)
p = pyaudio.PyAudio()

### Load some required pretrained files and define a plotting function to visualise the artist clusters

In [ ]:
encoder = VoiceEncoder()
reducer = pkl.load(open('reducer.sav','rb'))
knn = pkl.load(open('knn_model.sav','rb'))
projs = np.load('projections.npy')
labels = np.load('labels.npy')

singers = ['Arijit', 'Atif', 'Lata', 'Chinmayi']
cluster_coords = [(12.3,1.3),(12.3,13),(-4,-2),(-4,16)]

def plot_projection(projs_gt, labels_gt, projs_test):
  fig = plt.figure()
  ax = fig.add_subplot(111)
  ax.scatter(projs_gt[:,0],projs_gt[:,1],c=labels)

  for i in range(len(singers)):
    plt.annotate(singers[i],cluster_coords[i])

  for item in projs_test:
    ax.scatter(item[0],item[1],color='black',marker='+')

  plt.show()
  return

### Aaand, time to play!

In [ ]:
prediction_history=[]

Running below cell will record for 10 seconds from your mic, compare your voice to 4 popular singers that are currently stored in the system - Arijit Singh (red), Atif Aslam (blue), Lata Mangeshkar (green) and Chinmayi (cyan), and plot your voice as a 'x' mark along with the clusters corresponding to the artists in a 2-d space.

In [ ]:
print('Recording')
stream = p.open(format=FORMAT,channels=1,rate=RATE,input=True,frames_per_buffer=FRAMESIZE)
data = stream.read(NOFRAMES*FRAMESIZE)
decoded = np.frombuffer(data, dtype=np.int16)
stream.stop_stream()
stream.close()

embed = encoder.embed_utterance(decoded/np.max(decoded))
proj = reducer.transform([embed])
print('You sound most like: %s!'%singers[knn.predict(proj)[0]])
prediction_history.append([proj[0][0],proj[0][1]])
plot_projection(projs,labels,prediction_history)

Run above cell repeatedly to continue playing with this demo! One fun thing to try would be to check how your voice sample's embedding changes when you sound male versus when you sound female. Have fun! :D

In [ ]:
# when you're all done
p.terminate()
plt.close()